## 2026 EY AI & Data Challenge - Landsat Data Extraction Notebook

This notebook demonstrates Landsat data extraction and the creation of an output file to be used by the benchmark notebook. The baseline data is [Landsat Collection 2 Level 2](https://planetarycomputer.microsoft.com/dataset/landsat-c2-l2) data from the MS Planetary Computer catalog. 

<b>Caution</b> ... This notebook requires significant execution time as there are 9319 data points (unique locations and times) used for data extraction from the Landsat archive. The code takes about 7 hours to run to completion on a typical laptop computer and typical internet connection. Lower execution times are likely possible with optimization of the data extraction process and use of cloud computing services. 

### Load Python Dependencies

In [9]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt

# Data manipulation and analysis
import numpy as np
import pandas as pd

# Planetary Computer tools for STAC API access and authentication
import pystac_client
import planetary_computer as pc
from odc.stac import stac_load
from pystac.extensions.eo import EOExtension as eo

import tqdm

from datetime import date
from tqdm import tqdm
import os
import time

<h3>Extracting Landsat Data Using API Calls</h3> <p align="justify"> The API-based method allows us to efficiently access <b>Landsat</b> data for specific coordinates and time periods, ensuring scalability and reproducibility of the process. </p> <p align="justify"> Through the API, we can query individual bands or compute indices like <b>NDMI</b> on-the-fly. This approach reduces storage requirements and simplifies data preprocessing, making it ideal for large-scale environmental and water quality analysis. </p>

<p>The <b>compute_Landsat_values</b> function extracts Landsat surface reflectance values for specific sampling locations using a 100 m focal buffer around each point. For each location:</p>

<ul>
  <li>A bounding box (bbox) is created around the latitude and longitude coordinates.</li>
  <li>The Microsoft Planetary Computer API is queried for Landsat-8 Level-2 surface reflectance imagery within the date range.</li>
  <li>The nearest low-cloud (<10% cloud cover) scene is selected, and the specified bands (<b>green</b>, <b>nir08</b>, <b>swir16</b>, <b>swir22</b>) are loaded.</li>
  <li>Median values of the pixels within the bounding box are computed to reduce the effect of noise or outliers.</li>
</ul>

<p><b>Why the buffer value is 0.00089831:</b></p>

<p>We want a ~100 m buffer around each point. At the equator, 1 degree ≈ 110 km. Therefore, the degree equivalent of 100 m is:</p>

<p style="text-align:center;">
  <em>buffer_deg = 100 m / 110,000 m/deg ≈ 0.00089831</em>
</p>

<p>This slightly adjusted value ensures that the buffer approximately matches the pixel resolution of Landsat imagery, capturing a ~100 m area around each sampling location.</p>


In [10]:
import rasterio

# Configuração para tornar o GDAL mais resiliente a erros de rede
import os
os.environ["GDAL_HTTP_MAX_RETRY"] = "3"
os.environ["GDAL_HTTP_RETRY_DELAY"] = "5"
tqdm.pandas()

In [11]:
def compute_Landsat_values(row, max_retries=3):
    # Colunas de saída para o Machine Learning
    output_cols = [
        "w_coastal", "w_blue", "w_green", "w_red", "w_lwir11", "w_mndwi", "w_clorofila", "w_turbid_ndti",
        "l_nir", "l_swir16", "l_swir22", "l_ndvi", "l_ndmi", "platform", "diff_days", "collection"
    ]
    
    for attempt in range(max_retries):
        try:
            # 1. CONVERSÃO DE TIPOS (Evita o erro do np.float64)
            lat = float(row['Latitude'])
            lon = float(row['Longitude'])
            sample_dt = pd.to_datetime(row['Sample Date'], dayfirst=True, errors='coerce')
            
            if pd.isna(sample_dt):
                return pd.Series({k: np.nan for k in output_cols})

            # 2. CONFIGURAÇÃO DA GEOMETRIA E TEMPO
            # Usamos Point para a busca (mais preciso) e BBox para o download (área)
            geometry = {"type": "Point", "coordinates": [lon, lat]}
            
            bbox_size = 0.03
            bbox = [lon - bbox_size/2, lat - bbox_size/2, lon + bbox_size/2, lat + bbox_size/2]

            # Janela de +- 60 dias (Garante dados mesmo em buracos de cobertura de 2011)
            start_date = sample_dt - pd.Timedelta(days=60)
            end_date = sample_dt + pd.Timedelta(days=60)
            time_window = f"{start_date.strftime('%Y-%m-%d')}/{end_date.strftime('%Y-%m-%d')}"

            # 3. BUSCA "NUCLEAR" (Sem filtros restritivos)
            catalog = pystac_client.Client.open("https://planetarycomputer.microsoft.com/api/stac/v1")
            
            search = catalog.search(
                collections=["landsat-c2-l2", "landsat-c2-l1"], # Trazemos tudo o que tiver
                intersects=geometry, # Busca por Ponto é mais segura que BBox
                datetime=time_window
                # Sem query de nuvem ou plataforma aqui. Filtramos depois.
            )
            
            items = search.item_collection()
            
            if len(items) == 0:
                # Se falhar com Point, é porque realmente não existe nada num raio de 60 dias
                return pd.Series({k: np.nan for k in output_cols})

            # 4. ORDENAÇÃO E SELEÇÃO INTELIGENTE
            # Calculamos a distância em dias para cada item encontrado
            sample_date_utc = sample_dt.tz_localize("UTC") if sample_dt.tzinfo is None else sample_dt.tz_convert("UTC")
            
            # Ordena pelo item mais próximo temporalmente
            selected_item = sorted(items, key=lambda x: abs(x.datetime.replace(tzinfo=pd.Timestamp(x.datetime).tzinfo or "UTC") - sample_date_utc))[0]
            
            # Extração de Metadados
            platform = selected_item.properties.get("platform", "unknown")
            coll_id = "L2" if "c2-l2" in selected_item.collection_id else "L1"
            diff_days = abs((selected_item.datetime.replace(tzinfo=None) - sample_dt.replace(tzinfo=None)).days)

            # 5. MAPEAMENTO DE BANDAS (Correção Definitiva para Térmica)
            assets = selected_item.assets.keys()
            mapping = {
                "blue": "blue", "green": "green", "red": "red",
                "swir16": "swir16", "swir22": "swir22", "qa_pixel": "qa_pixel"
            }
            
            # --- CORREÇÃO DA BANDA NIR ---
            # Se não tiver nir08 (L8), tenta nir (L5/L7)
            mapping["nir"] = "nir08" if "nir08" in assets else "nir"

            # --- CORREÇÃO DA BANDA TÉRMICA ---
            # Procuramos qual nome de banda térmica existe neste item específico
            # st_b6 = Surface Temperature (Landsat 5/7 Level 2)
            # lwir11 = Landsat 8/9
            # lwir = Landsat 5/7 Level 1
            thermal_candidates = ["lwir11", "st_b6", "lwir", "lwir60"]
            
            thermal_band_found = None
            for cand in thermal_candidates:
                if cand in assets:
                    thermal_band_found = cand
                    break
            
            if thermal_band_found:
                mapping["lwir11"] = thermal_band_found
            else:
                # Se não achar nenhuma térmica, removemos do mapping para não quebrar o download
                # O valor ficará NaN depois
                pass

            # Banda Coastal (Só existe no L8/L9)
            if "coastal" in assets: mapping["coastal"] = "coastal"

            # 6. DOWNLOAD (STAC LOAD)
            # Passamos apenas as bandas que realmente existem no mapping
            bands_to_load = list(mapping.values())
            
            data = stac_load(
                [selected_item], 
                bands=bands_to_load, 
                bbox=bbox, 
                patch_url=pc.sign_url, 
                crs="EPSG:4326", 
                resolution=0.00027
            ).compute().isel(time=0)

            # 7. ESCALONAMENTO E CORREÇÃO DE VALORES
            bands_spectral = [b for b in ["blue", "green", "red", "nir", "swir16", "swir22", "coastal"] if b in mapping]
            
            if coll_id == "L2":
                # Fatores oficiais para Collection 2 Level 2 (Refletância)
                for b in bands_spectral:
                    if mapping[b] in data: # Segurança extra
                         data[mapping[b]] = (data[mapping[b]] * 0.0000275) - 0.2
                
                # Térmica L2 (Seja lwir11 ou st_b6, a fórmula Kelvin é a mesma na Collection 2)
                if "lwir11" in mapping and mapping["lwir11"] in data:
                    data[mapping["lwir11"]] = (data[mapping["lwir11"]] * 0.00341802 + 149.0) - 273.15
                
            else: # Level 1 (Fallback)
                for b in bands_spectral:
                    if mapping[b] in data:
                        data[mapping[b]] = data[mapping[b]] / 65535.0
                
                if "lwir11" in mapping and mapping["lwir11"] in data:
                     data[mapping["lwir11"]] = (data[mapping["lwir11"]] * 0.00341802 + 149.0) - 273.15
                     
            # 8. MÁSCARAS E ÍNDICES
            
            qa = data[mapping["qa_pixel"]]

            # A. Definição do que é "Ruim" (Nuvens, Sombras e Bordas)
            # Bit 1: Dilated Cloud (Bordas)
            # Bit 3: Cloud (Nuvem)
            # Bit 4: Cloud Shadow (Sombra)
            # Usamos a operação bitwise OR (|) para juntar todos os problemas
            cloud_mask = (qa & (1 << 3)) > 0
            shadow_mask = (qa & (1 << 4)) > 0
            dilated_mask = (qa & (1 << 1)) > 0
            
            # Se for L8 ou L9, removemos também Cirrus (Bit 2)
            cirrus_mask = (qa & (1 << 2)) > 0 if "nir08" in mapping.values() else False
            
            # Máscara de Exclusão Total: Qualquer pixel que tenha um desses problemas
            all_bad_pixels = cloud_mask | shadow_mask | dilated_mask | cirrus_mask

            # B. Definição do que é Água (Candidatos)
            # Usamos Bit 7 (Water) OU MNDWI > 0 (para pegar rios que o QA perdeu)
            qa_is_water = (qa & (1 << 7)) > 0
            
            # Calculamos MNDWI com segurança (evitando divisão por zero)
            # Adicionamos um valor ínfimo (1e-6) para não dar erro se a soma for 0
            mndwi_num = data[mapping["green"]] - data[mapping["swir16"]]
            mndwi_den = data[mapping["green"]] + data[mapping["swir16"]] + 1e-6
            mndwi = mndwi_num / mndwi_den
            
            # Candidato a água: MNDWI positivo ou QA diz que é água
            water_candidate = (mndwi > 0.0) | qa_is_water

            # C. FILTRAGEM FINAL (A Água Limpa)
            # É água E NÃO TEM nuvem/sombra
            is_water_clean = water_candidate & (~all_bad_pixels)
            
            # D. FILTRAGEM FINAL (A Terra Limpa)
            # Não é água E NÃO TEM nuvem/sombra
            is_land_clean = (~water_candidate) & (~all_bad_pixels)

            # E. Cálculo dos Índices (Só calculamos, o filtro vem na média)
            # Adicionamos 1e-6 para evitar divisão por zero
            ndvi = (data[mapping["nir"]] - data[mapping["red"]]) / (data[mapping["nir"]] + data[mapping["red"]] + 1e-6)
            ndmi = (data[mapping["nir"]] - data[mapping["swir16"]]) / (data[mapping["nir"]] + data[mapping["swir16"]] + 1e-6)
            ndti = (data[mapping["red"]] - data[mapping["green"]]) / (data[mapping["red"]] + data[mapping["green"]] + 1e-6)
            clorofila = data[mapping["nir"]] / (data[mapping["blue"]] + 1e-6)
            
            # 9. EXTRAÇÃO DE MÉDIAS
            def get_mean(da, mask):
                # Aplica a máscara: Onde a máscara for False, vira NaN
                vals = da.where(mask)
                
                # Conta quantos pixels válidos sobraram
                valid_pixels = vals.count().item()
                
                # Se não sobrou nenhum pixel (ex: dia totalmente nublado em cima do rio), retorna NaN
                if valid_pixels == 0:
                    return np.nan
                
                return float(vals.mean().item())

            return pd.Series({
                # Features de Água (Usando is_water_clean)
                "w_coastal": get_mean(data[mapping["coastal"]], is_water_clean) if "coastal" in mapping else np.nan,
                "w_blue": get_mean(data[mapping["blue"]], is_water_clean),
                "w_green": get_mean(data[mapping["green"]], is_water_clean),
                "w_red": get_mean(data[mapping["red"]], is_water_clean),
                "w_lwir11": get_mean(data[mapping["lwir11"]], is_water_clean),
                "w_mndwi": get_mean(mndwi, is_water_clean),
                "w_clorofila": get_mean(clorofila, is_water_clean),
                "w_turbid_ndti": get_mean(ndti, is_water_clean),
                
                # Features de Terra (Usando is_land_clean)
                "l_nir": get_mean(data[mapping["nir"]], is_land_clean),
                "l_swir16": get_mean(data[mapping["swir16"]], is_land_clean),
                "l_swir22": get_mean(data[mapping["swir22"]], is_land_clean),
                "l_ndvi": get_mean(ndvi, is_land_clean),
                "l_ndmi": get_mean(ndmi, is_land_clean),
                
                # Metadados
                "platform": platform,
                "diff_days": diff_days,
                "collection": coll_id,
                "pixels_water": int(is_water_clean.sum().item()) # Útil para saber se pegou água suficiente
            })

        except Exception as e:
            # Tratamento de erro 403/429 (Rate Limit)
            if "403" in str(e) or "429" in str(e):
                time.sleep(10) # Espera maior se for rate limit
                continue
            elif attempt < max_retries:
                time.sleep( attempt* 2**3) # Espera antes de tentar novamente
                continue
            
            print(f"Erro Linha {row.name} (Lat:{row['Latitude']}, Lon:{row['Longitude']}): {e}")
            return pd.Series({k: np.nan for k in output_cols})

    return pd.Series({k: np.nan for k in output_cols})

### Extracting features for the training dataset

In [12]:
Water_Quality_df=pd.read_csv('water_quality_training_dataset.csv')
Water_Quality_df.head()

,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-28.760833,17.730278,02-01-2011,128.912,555.0,10.0
1,-26.861111,28.884722,03-01-2011,74.720,162.9,163.0
2,-26.450000,28.085833,03-01-2011,89.254,573.0,80.0
3,-27.671111,27.236944,03-01-2011,82.000,203.6,101.0
4,-27.356667,27.286389,03-01-2011,56.100,145.1,151.0


In [13]:
Water_Quality_df.shape

(9319, 6)

In [14]:
Water_Quality_df_10 = Water_Quality_df.loc[665:677]
Water_Quality_df_200 = Water_Quality_df.loc[0:199]
Water_Quality_df_200.shape

(200, 6)

<h3>Note:</h3>
<p>The Landsat data extraction process for all 9,319 locations typically requires 7+ hours when executed in a single run. During long executions, you may occasionally encounter API limits, timeout errors, or request failures. To avoid these interruptions, we recommend running the extraction in smaller batches.</p>

<p>In this notebook, we provide a sample code snippet demonstrating how to extract data for the first 200 locations. Participants are encouraged to follow the same batching approach to extract data for all 9,319 locations safely and efficiently.</p>

<p>We have already executed the full extraction for all 9,319 locations and saved the output to <b>landsat_features_training.csv</b>, which will be used in the benchmark notebook.
Similarly, participants can extract Landsat features in batches, combine the batch outputs, and save the final merged dataset as <b>landsat_features_training.csv</b> to ensure the benchmark notebook runs smoothly.</p>

In [7]:
# Extract band values from Landsat for training dataset
train_features_path = "landsat_features_training_10.csv"

print("🚀 Running Landsat feature extraction for training data...")
landsat_train_features = Water_Quality_df_10.progress_apply(compute_Landsat_values, axis=1)
landsat_train_features.to_csv(train_features_path, index=False)

🚀 Running Landsat feature extraction for training data...


100%|██████████| 13/13 [01:19<00:00,  6.15s/it]


In [ ]:
train_features_path = "../Datasets/landsat_features_more_bands_separated_training.csv"
batch_size = 300
total_rows = len(Water_Quality_df)

print(f"🚀 Iniciando extração total de {total_rows} linhas em lotes de {batch_size}...")

# Se quiser começar do zero, remove o arquivo antigo
if os.path.exists(train_features_path):
    os.remove(train_features_path)

# Loop pelos índices (sem tqdm aqui no range, para não duplicar barras)
for i in range(0, total_rows, batch_size):
    
    # 1. Seleciona o lote
    batch = Water_Quality_df.iloc[i : i + batch_size]
    
    print(f"\nProcessing Batch {i // batch_size + 1} / {(total_rows // batch_size) + 1} (Rows {i} to {min(i + batch_size, total_rows)})...")
    
    # 2. Aplica a função com progress_apply (Isso mostra a barra por LINHA dentro do lote)
    batch_results = batch.progress_apply(compute_Landsat_values, axis=1)
    
    # 3. Salva no CSV
    # Se for o primeiro lote (i=0), escreve ('w') com cabeçalho
    # Se forem os próximos, anexa ('a') sem cabeçalho
    if i == 0:
        batch_results.to_csv(train_features_path, mode='w', index=False, header=True)
    else:
        batch_results.to_csv(train_features_path, mode='a', index=False, header=False)

print("\n✅ Processamento concluído!")


🚀 Iniciando extração total de 9319 linhas em lotes de 300...

Processing Batch 1 / 32 (Rows 0 to 300)...


 10%|█         | 30/300 [01:45<14:23,  3.20s/it]

In [27]:
train_features_path = "../Datasets/landsat_features_more_bands_training.csv"
chunk_size = 600

print(f"🚀 Iniciando extração de {len(Water_Quality_df)} linhas em blocos de {chunk_size}...")

# O loop percorre o dataframe de 600 em 600
for i in range(0, len(Water_Quality_df), chunk_size):
    # Seleciona o bloco atual
    subset = Water_Quality_df.iloc[i : i + chunk_size]
    
    print(f"\n📦 Processando bloco {i//chunk_size + 1} (linhas {i} até {i + len(subset) - 1})...")
    
    # Aplica a função no bloco atual
    # Nota: usamos axis=1 para passar a linha inteira para a função
    chunk_features = subset.progress_apply(compute_Landsat_values, axis=1)
    
    # Verifica se o arquivo já existe para decidir se escreve o cabeçalho (header)
    file_exists = os.path.isfile(train_features_path)
    
    # Salva o bloco atual no CSV (mode='a' é para dar APPEND, ou seja, anexar ao final)
    chunk_features.to_csv(
        train_features_path, 
        mode='a', 
        index=False, 
        header=not file_exists, # Só coloca o cabeçalho se o arquivo estiver sendo criado agora
        encoding='utf-8'
    )
    
    print(f"✅ Bloco {i//chunk_size + 1} salvo com sucesso em: {train_features_path}")
    time.sleep(16)  # Pequena pausa entre blocos para evitar sobrecarga

print("\n✨ Processamento concluído!")

🚀 Iniciando extração de 9319 linhas em blocos de 600...

📦 Processando bloco 1 (linhas 0 até 599)...


100%|██████████| 600/600 [28:34<00:00,  2.86s/it]


✅ Bloco 1 salvo com sucesso em: ../Datasets/landsat_features_more_bands_training.csv

📦 Processando bloco 2 (linhas 600 até 1199)...


100%|██████████| 600/600 [30:55<00:00,  3.09s/it]


✅ Bloco 2 salvo com sucesso em: ../Datasets/landsat_features_more_bands_training.csv

📦 Processando bloco 3 (linhas 1200 até 1799)...


...

✅ Bloco 14 salvo com sucesso em: ../Datasets/landsat_features_more_bands_training.csv

📦 Processando bloco 15 (linhas 8400 até 8999)...


100%|██████████| 600/600 [35:45<00:00,  3.58s/it]


✅ Bloco 15 salvo com sucesso em: ../Datasets/landsat_features_more_bands_training.csv

📦 Processando bloco 16 (linhas 9000 até 9318)...


100%|██████████| 319/319 [18:50<00:00,  3.54s/it]


✅ Bloco 16 salvo com sucesso em: ../Datasets/landsat_features_more_bands_training.csv

✨ Processamento concluído!


<p><b>NDMI and MNDWI Indices:</b></p>
<p>In this notebook, we compute two commonly used water-related indices from the extracted Landsat bands:</p>
<ul>
  <li><b>NDMI (Normalized Difference Moisture Index):</b> Measures vegetation water content and surface moisture. Computed as <em>(NIR - SWIR16) / (NIR + SWIR16)</em>.</li>
  <li><b>MNDWI (Modified Normalized Difference Water Index):</b> Highlights open water features by enhancing water reflectance and suppressing built-up areas. Computed as <em>(Green - SWIR16) / (Green + SWIR16)</em>.</li>
</ul>

<p>An <b>epsilon value</b> (<em>eps = 1e-10</em>) is added in the denominators to avoid division by zero. These indices are widely used in hydrological and water quality analyses for detecting water presence and vegetation moisture levels.</p>


In [29]:
landsat_train_features = pd.read_csv(train_features_path)
landsat_train_features.shape

(9319, 8)

In [30]:
# Create indices: NDMI and MNDWI
eps = 1e-10
landsat_train_features['NDMI'] = (landsat_train_features['nir'] - landsat_train_features['swir16']) / (landsat_train_features['nir'] + landsat_train_features['swir16'] + eps)
landsat_train_features['MNDWI'] = (landsat_train_features['green'] - landsat_train_features['swir16']) / (landsat_train_features['green'] + landsat_train_features['swir16'] + eps)
landsat_train_features['Clorfilia'] = (landsat_train_features['green']/landsat_train_features['blue'] + eps)
landsat_train_features['Turbidity'] = (landsat_train_features['red']/landsat_train_features['blue'] + eps)
landsat_train_features['NDTI'] = (landsat_train_features['red'] - landsat_train_features['green']) / (landsat_train_features['red'] + landsat_train_features['green'] + eps)
landsat_train_features["NDVI"] = (landsat_train_features['nir'] - landsat_train_features['red']) / (landsat_train_features['nir'] + landsat_train_features['red'] + eps)

In [31]:
landsat_train_features['Latitude'] = Water_Quality_df['Latitude']
landsat_train_features['Longitude'] = Water_Quality_df['Longitude']
landsat_train_features['Sample Date'] = Water_Quality_df['Sample Date']
landsat_train_features = landsat_train_features[['Latitude', 'Longitude', 'Sample Date', 'nir', 'green', 'swir16', 'swir22', 'coastal',
                                                 'blue', 'red', 'lwir11', 'NDMI', 'MNDWI', 'Clorfilia', 'Turbidity', 'NDTI', 'NDVI']]

In [32]:
final_train_features_path = "../Datasets/landsat_features_more_bands_final_training.csv"
landsat_train_features.to_csv(final_train_features_path, index=False)

In [10]:
# Preview File
landsat_train_features.head()

,Latitude,Longitude,Sample Date,nir,green,swir16,swir22,NDMI,MNDWI
0,-28.760833,17.730278,02-01-2011,11190.0,11426.0,7687.5,7645.0,0.185538,0.195595
1,-26.861111,28.884722,03-01-2011,17658.5,9550.0,13746.5,10574.0,0.124566,-0.180134
2,-26.450000,28.085833,03-01-2011,15210.0,10720.0,17974.0,14201.0,-0.083293,-0.252805
3,-27.671111,27.236944,03-01-2011,14887.0,10943.0,13522.0,11403.0,0.048048,-0.105416
4,-27.356667,27.286389,03-01-2011,16828.5,9502.5,12665.5,9643.0,0.141147,-0.142683


### Joining data from the example data set with the downloaded dataset

In [42]:
landsat_example_train_features = pd.read_csv('../Datasets/landsat_features_training.csv')
landsat_train_features['Sample Date'] = pd.to_datetime(landsat_train_features['Sample Date'], dayfirst=True, errors='coerce')
landsat_example_train_features['Sample Date'] = pd.to_datetime(landsat_example_train_features['Sample Date'], dayfirst=True, errors='coerce')

# 2. Definir as colunas que servem de "ID" (as chaves)
chaves = ['Latitude', 'Longitude', 'Sample Date']

# 3. Fazer o merge (Outer Join)
# how='outer' garante que:
# - Se a linha só existe no seu, ela fica.
# - Se a linha só existe no exemplo, ela fica.
# - Se existe nos dois, elas se juntam.
df_merged = pd.merge(
    landsat_train_features, 
    landsat_example_train_features, 
    on=chaves, 
    how='outer', 
    suffixes=('', '_exemplo') # Adiciona sufixo nas colunas repetidas (nir, green, etc)
)

# 4. Lista das colunas que existem nos dois datasets
colunas_comuns = ['nir', 'green', 'swir16', 'swir22', 'NDMI', 'MNDWI']

# 5. Preencher os vázios: se a coluna original for NaN, pega o valor da coluna _exemplo
for col in colunas_comuns:
    df_merged[col] = df_merged[col].fillna(df_merged[col + '_exemplo'])

df_merged = df_merged.sort_values(by=['Sample Date', 'Latitude', 'Longitude'])
df_merged['Sample Date'] = df_merged['Sample Date'].dt.strftime('%d-%m-%Y')


# 6. Agora que preenchemos, podemos apagar as colunas auxiliares do exemplo
colunas_para_remover = [col + '_exemplo' for col in colunas_comuns]
df_merged = df_merged.drop(columns=colunas_para_remover)

# 7. Verificar o resultado
print(f"Linhas no seu original: {len(landsat_train_features)}")
print(f"Linhas no exemplo: {len(landsat_example_train_features)}")
print(f"Linhas após a união: {len(df_merged)}")

# Salvar o resultado final
df_merged.to_csv("../Datasets/landsat_features_more_bands_train_merged.csv", index=False)


Linhas no seu original: 9319
Linhas no exemplo: 9319
Linhas após a união: 9319


### Extracting features for the validation dataset

In [43]:
Validation_df=pd.read_csv('submission_template.csv')
Validation_df.head()

,Latitude,Longitude,Sample Date,Total Alkalinity,Electrical Conductance,Dissolved Reactive Phosphorus
0,-32.043333,27.822778,01-09-2014,NaN,NaN,NaN
1,-33.329167,26.077500,16-09-2015,NaN,NaN,NaN
2,-32.991639,27.640028,07-05-2015,NaN,NaN,NaN
3,-34.096389,24.439167,07-02-2012,NaN,NaN,NaN
4,-32.000556,28.581667,01-10-2014,NaN,NaN,NaN


In [44]:
Validation_df.shape

(200, 6)

In [13]:
# Extract band values from Landsat for submission dataset
val_features_path = "landsat_features_validation.csv"

print("🚀 Running Landsat feature extraction for validation data...")
landsat_val_features = Validation_df.progress_apply(compute_Landsat_values, axis=1)
landsat_val_features.to_csv(val_features_path, index=False)

🚀 Running Landsat feature extraction for validation data...


100%|██████████| 200/200 [10:38<00:00,  3.19s/it]


In [50]:
val_features_path = "../Datasets/landsat_features_more_bands_validation.csv"

print("🚀 Running Landsat feature extraction for validation data with more bands...")
landsat_val_features = Validation_df.progress_apply(compute_Landsat_values, axis=1)
landsat_val_features.to_csv(val_features_path, index=False)

🚀 Running Landsat feature extraction for validation data with more bands...


100%|██████████| 200/200 [13:29<00:00,  4.05s/it]


In [51]:
# Create indices: NDMI and MNDWI
eps = 1e-10
landsat_val_features['NDMI'] = (landsat_val_features['nir'] - landsat_val_features['swir16']) / (landsat_val_features['nir'] + landsat_val_features['swir16'] + eps)
landsat_val_features['MNDWI'] = (landsat_val_features['green'] - landsat_val_features['swir16']) / (landsat_val_features['green'] + landsat_val_features['swir16'] + eps)
landsat_val_features['Clorfilia'] = (landsat_val_features['green']/landsat_val_features['blue'] + eps)
landsat_val_features['Turbidity'] = (landsat_val_features['red']/landsat_val_features['blue'] + eps)
landsat_val_features['NDTI'] = (landsat_val_features['red'] - landsat_val_features['green']) / (landsat_val_features['red'] + landsat_val_features['green'] + eps)
landsat_val_features["NDVI"] = (landsat_val_features['nir'] - landsat_val_features['red']) / (landsat_val_features['nir'] + landsat_val_features['red'] + eps)
landsat_val_features.to_csv(val_features_path, index=False)
landsat_val_features

,nir,green,swir16,swir22,coastal,blue,red,lwir11,NDMI,MNDWI,Clorfilia,Turbidity,NDTI,NDVI
0,15229.0,12868.0,14797.0,12421.0,10961.0,11282.0,13210.0,39652.5,0.014388,-0.069727,1.140578,1.170892,0.013115,0.070994
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15081.0,9472.5,11916.0,9558.5,NaN,8576.5,9311.0,NaN,0.117235,-0.114244,1.104472,1.085641,-0.008598,0.236553
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9125.0,11100.5,9455.0,8711.0,8926.0,9504.0,11166.0,44180.5,-0.017761,0.080052,1.167982,1.174874,0.002942,-0.100586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,20187.0,15329.0,18460.5,16786.0,NaN,14179.0,15939.0,NaN,0.044673,-0.092677,1.081106,1.124127,0.019509,0.117588
196,15883.0,9083.5,12135.5,9484.0,7758.0,8045.0,8931.5,42839.5,0.133751,-0.143833,1.129086,1.110193,-0.008437,0.280139
197,13619.5,10046.5,13105.0,10969.0,8424.0,8825.5,10108.0,45369.0,0.019252,-0.132108,1.138349,1.145318,0.003051,0.147993
198,13955.5,10670.0,17303.5,14835.5,9074.5,9505.0,11638.0,49183.5,-0.107105,-0.237135,1.122567,1.224408,0.043393,0.090550


In [55]:
landsat_val_features['Latitude'] = Validation_df['Latitude']
landsat_val_features['Longitude'] = Validation_df['Longitude']
landsat_val_features['Sample Date'] = Validation_df['Sample Date']
landsat_val_features = landsat_val_features[['Latitude', 'Longitude', 'Sample Date', 'nir', 'green', 'swir16', 'swir22', 'coastal',
                                                 'blue', 'red', 'lwir11', 'NDMI', 'MNDWI', 'Clorfilia', 'Turbidity', 'NDTI', 'NDVI']]

In [56]:
landsat_val_features.to_csv(val_features_path, index=False)

In [57]:
# Preview File
landsat_val_features.head()

,Latitude,Longitude,Sample Date,nir,green,swir16,swir22,coastal,blue,red,lwir11,NDMI,MNDWI,Clorfilia,Turbidity,NDTI,NDVI
0,-32.043333,27.822778,01-09-2014,15229.0,12868.0,14797.0,12421.0,10961.0,11282.0,13210.0,39652.5,0.014388,-0.069727,1.140578,1.170892,0.013115,0.070994
1,-33.329167,26.077500,16-09-2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-32.991639,27.640028,07-05-2015,15081.0,9472.5,11916.0,9558.5,NaN,8576.5,9311.0,NaN,0.117235,-0.114244,1.104472,1.085641,-0.008598,0.236553
3,-34.096389,24.439167,07-02-2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-32.000556,28.581667,01-10-2014,9125.0,11100.5,9455.0,8711.0,8926.0,9504.0,11166.0,44180.5,-0.017761,0.080052,1.167982,1.174874,0.002942,-0.100586


## joining the new features with the example ones

In [58]:
landsat_example_val_features = pd.read_csv('../Datasets/landsat_features_validation.csv')


landsat_val_features['Sample Date'] = pd.to_datetime(landsat_val_features['Sample Date'], dayfirst=True, errors='coerce')
landsat_example_val_features['Sample Date'] = pd.to_datetime(landsat_example_val_features['Sample Date'], dayfirst=True, errors='coerce')

# 2. Definir as colunas que servem de "ID" (as chaves)
chaves = ['Latitude', 'Longitude', 'Sample Date']

# 3. Fazer o merge (Outer Join)
# how='outer' garante que:
# - Se a linha só existe no seu, ela fica.
# - Se a linha só existe no exemplo, ela fica.
# - Se existe nos dois, elas se juntam.
df_merged = pd.merge(
    landsat_val_features, 
    landsat_example_val_features, 
    on=chaves, 
    how='outer', 
    suffixes=('', '_exemplo') # Adiciona sufixo nas colunas repetidas (nir, green, etc)
)

# 4. Lista das colunas que existem nos dois datasets
colunas_comuns = ['nir', 'green', 'swir16', 'swir22', 'NDMI', 'MNDWI']

# 5. Preencher os vázios: se a coluna original for NaN, pega o valor da coluna _exemplo
for col in colunas_comuns:
    df_merged[col] = df_merged[col].fillna(df_merged[col + '_exemplo'])

df_merged = df_merged.sort_values(by=['Sample Date', 'Latitude', 'Longitude'])
df_merged['Sample Date'] = df_merged['Sample Date'].dt.strftime('%d-%m-%Y')


# 6. Agora que preenchemos, podemos apagar as colunas auxiliares do exemplo
colunas_para_remover = [col + '_exemplo' for col in colunas_comuns]
df_merged = df_merged.drop(columns=colunas_para_remover)

# 7. Verificar o resultado
print(f"Linhas no seu original: {len(landsat_val_features)}")
print(f"Linhas no exemplo: {len(landsat_example_val_features)}")
print(f"Linhas após a união: {len(df_merged)}")

# Salvar o resultado final
df_merged.to_csv("../Datasets/landsat_features_more_bands_val_merged.csv", index=False)

Linhas no seu original: 200
Linhas no exemplo: 200
Linhas após a união: 200
